## 1. Cartpole 강화학습 준비

In [54]:
# import
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

### 1.1 Cartpole 시연함수

In [55]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frame(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)   # 화면크기, 해상도 설정
    patch = plt.imshow(frames[0])   # 첫번째 프레임을 화면에 표시
    plt.axis('off')   # 축 표시 안함

    def animate(i):
        patch.set_data(frames[i])   # i번째 프레임 업데이트

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)   # 애니메이션 생성

    anim.save('cartpole_DQN.gif')  # 시연영상을 저장
    display(display_animation(anim, default_mode='loop'))

### 1.2 Constant 정의

In [56]:
#name tuple을 사용하여 Transition 정의
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))

In [57]:
# 상수 정의
ENV = 'CartPole-v0'  # task
GAMMA = 0.99  # discount factor
MAX_STEPS = 200  # 1에피소드 당 최대 단계 수
NUM_EPISODES = 500  # 최대 에피소드 수

## 2. ReplayMemory

In [58]:
class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # 메모리의 최대 저장 건수
        self.memory = []  # 실제 transition을 저장할 변수
        self.index = 0  # 저장 위치를 가리킬 인덱스 변수

    def push(self, state, action, state_next, reward):

        # 메모리 공간 확인
        if len(self.memory) < self.capacity:
            self.memory.append(None) 

        self.memory[self.index] = Transition(state, action, state_next, reward)  # 새로운 transition을 메모리에 추가

        self.index = (self.index + 1) % self.capacity   # 인덱스 설정

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)   # 메모리에서 랜덤하게 배치 추출

    def __len__(self):
        return len(self.memory)   # 메모리 크기 반환

In [59]:
#!pip install --upgrade pip

In [60]:
#!pip install torch

## 3. Cartpole DQN 설계

In [61]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32   # 배치 크기 설정
CAPACITY = 10000   # 메모리 용량 설정


class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  
        
        # 리플레이 메모리 설정
        self.memory = ReplayMemory(CAPACITY)

        # 신경망 구성, 입력차원은 num_statees, 출력차원은 num_actions
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))   # 입력 num_stage, 출력 32 인 선형 변환 수행
        self.model.add_module('relu1', nn.ReLU())     # 활성화 함수로 비선형성 도입
        self.model.add_module('fc2', nn.Linear(32, 32))   # 입력 32, 출력 32인 선형 변환 수행
        self.model.add_module('relu2', nn.ReLU())     # 활성화 함수로 비선형성 도입
        self.model.add_module('fc3', nn.Linear(32, num_actions))   # 입력 32, 출력 num_actions 인 선형 변환 수행

        print(self.model)  # 신경망 구조 출력

        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)  # 옵티마이저 설정

    def replay(self):

        if len(self.memory) < BATCH_SIZE:
            return

        # 리플레이 버퍼에서 샘플링
        transitions = self.memory.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)  # state의 텐서
        action_batch = torch.cat(batch.action)  # action의 텐서
        reward_batch = torch.cat(batch.reward)  # reward의 텐서
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])   # none을 제외한 다음 state의 텐서

        self.model.eval()  # evaluation mode 로 전환

        state_action_values = self.model(state_batch).gather(1, action_batch)  # 실제 Q-value 계산 (target value)

        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        next_state_values = torch.zeros(BATCH_SIZE)
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        expected_state_action_values = reward_batch + GAMMA * next_state_values  # 예측 Q-value 계산 (predicted value)

        self.model.train()  # training mode 로 전환

        # 손실 계산
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        self.optimizer.zero_grad()  # 경사를 초기화
        loss.backward()  # 역전파 계산
        self.optimizer.step()  # 가중치 업데이트

    def decide_action(self, state, episode):
        epsilon = 0.5 * (1 / (episode + 1))

        # 입실론-그리디 전략으로 action 선택
        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # evaluation mode 로 전환
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)

        else:
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])   # 무작위 action 선택

        return action

## 4. Cartpole Agent 설계

In [62]:
class Agent:
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)
        
    def update_q_function(self):
        self.brain.replay()   # replay 메소드 호출

    def get_action(self, state, episode):
        action = self.brain.decide_action(state, episode)  # 현재 state에서의 action 선택
        return action

    def memorize(self, state, action, state_next, reward):
        self.brain.memory.push(state, action, state_next, reward)   # brain의 메모리에 transition을 저장

## 5. Cartpole 환경 설계

In [63]:
class Environment:

    def __init__(self):
        self.env = gym.make(ENV, render_mode="rgb_array") 
        num_states = self.env.observation_space.shape[0]  
        num_actions = self.env.action_space.n 
        self.agent = Agent(num_states, num_actions) 

        
    def run(self):
        episode_10_list = np.zeros(10)      # 최근 10 에피소드의 step 수 기록 리스트
        complete_episodes = 0      # 성공 에피소드 수
        episode_final = False       # 마지막 에피소드인지 확인
        frames = []     # 애니메이션 프레임 저장

        for episode in range(NUM_EPISODES):  
            observation = self.env.reset()  # 환경 리셋

            state = observation     # observation 저장
            state=state[0]
            state = torch.from_numpy(state).type(
                torch.FloatTensor)     # 텐서로 변환
            state = torch.unsqueeze(state, 0)    # state 텐서 차원 추가

            for step in range(MAX_STEPS): 

                if episode_final is True: 
                    frames.append(self.env.render())   # 마지막 에피소드면 렌더링

                action = self.agent.get_action(state, episode)      # agent가 action 선택

                observation_next, _, done, _,_ = self.env.step(
                    action.item())   # action 수행 후 결과 저장

                if done:      # 에피소드 종료
                    state_next = None  

                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))  # 가장 최근 10개 에피소드의 step 기록

                    if step < 190:   # step이 100미만
                        reward = torch.FloatTensor(
                            [-1.0])     # reward -1
                        complete_episodes = 0    # 연속 성공 에피소드 초기화
                    else:
                        reward = torch.FloatTensor([1.0])       # reward +1
                        complete_episodes = complete_episodes + 1  
                        print(complete_episodes)# 연속 성공 에피소드 증가
                else:
                    reward = torch.FloatTensor([0.0])    # 중간 reward 0 부여
                    state_next = observation_next  
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor) 
                    state_next = torch.unsqueeze(state_next, 0)  

                self.agent.memorize(state, action, state_next, reward)   # 메모리에 transition 저장

                self.agent.update_q_function()    # 신경망 업데이트

                state = state_next

                if done:
                    print('%d Episode: Finished after %d steps：최근 10 에피소드의 평균 단계 수 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                display_frame(frames)  # 애니메이션 저장, 표시
                break

            if complete_episodes >= 10:
                print('----- 10 episode success -----')
                episode_final = True

## 6. Cartpole DQN 학습

In [64]:
cartpole_env = Environment()
cartpole_env.run()

C:\Users\yejun\PycharmProjects\pythonProject\.venv\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
C:\Users\yejun\PycharmProjects\pythonProject\.venv\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
C:\Users\yejun\AppData\Local\Temp\ipykernel_17572\2177037993.py:52: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:28.)
  next_state_values[non_final_mask] = self.model(


Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 35 steps：최근 10 에피소드의 평균 단계 수 = 3.5
1 Episode: Finished after 27 steps：최근 10 에피소드의 평균 단계 수 = 6.2
2 Episode: Finished after 44 steps：최근 10 에피소드의 평균 단계 수 = 10.6
3 Episode: Finished after 29 steps：최근 10 에피소드의 평균 단계 수 = 13.5
4 Episode: Finished after 95 steps：최근 10 에피소드의 평균 단계 수 = 23.0
5 Episode: Finished after 43 steps：최근 10 에피소드의 평균 단계 수 = 27.3
6 Episode: Finished after 92 steps：최근 10 에피소드의 평균 단계 수 = 36.5
7 Episode: Finished after 63 steps：최근 10 에피소드의 평균 단계 수 = 42.8
8 Episode: Finished after 45 steps：최근 10 에피소드의 평균 단계 수 = 47.3
9 Episode: Finished after 55 steps：최근 10 에피소드의 평균 단계 수 = 52.8
10 Episode: Finished after 13 steps：최근 10 에피소드의 평균 단계 수 = 50.6
11 Episode: Finished after 20 steps：최근 10 에피소드의 평균 단계 수 = 49.9
12 Episode: Finished after 1